This notebook is written by the DAWS2 UQ team based in Univ. of Liverpool for the workshop of UQ & M of the DAWS2 project.

Uncertainty Quantification & Management
--
*placeholder texts*

{

Personal website: https://yuchenakaleslie.github.io/<br>
ResearchGate: https://www.researchgate.net/profile/Yu-Chen-402<br>
Contact: yu.chen2@liverpool.ac.uk

}

DAWS2 UQ team: {Scott Ferson, Ioanna,  [(Leslie) Yu Chen](https://yuchenakaleslie.github.io/)}

All rights reserved.

===================

## Table of Contents
#### [1. Setup](#Setup)
#### [2. Problem statement](#problem_statement)
#### [3. Create propagating function](#create_propagating_function)
#### [4. Epistemic uncertainty](#propagate_epistemic_uncertainty)
#### <small>[4.1. Uncertainty Characterisation](#4.1-uncertainty-characterisation)</small>
#### <small>[4.2. Uncertainty propagation methods](#4.2-uncertainty-propatation-methods)</small>
##### <small>[4.2.1. Endpoints propagation](#4.2.1-Endpoints-propatation)</small>
##### <small>[4.2.2. Subinterval reconstitution propagation](#4.2.2.-Subinterval-reconstitution-propagation)</small>
##### <small>[4.2.3. Sampling propagation](#4.2.3.-Sampling-propagation)</small>
##### <small>[4.2.4. Optimisation methods](#4.2.3.-Optimisation-methods)</small>
#### [5. Aleatory uncertainty](#aleatory_uncertainty_propagation)
#### [5. Mixed types of uncertainty](#mixed_uncertainty_propagation)

***
<a id="setup"></a>
# 1. Setup

Import the libraries

In [9]:
import numpy as np
from PyUncertainNumber.UP.uncertainty_propagation import up_bb
from PyUncertainNumber import UncertainNumber as UN

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext rich

In [3]:
# import user-defined function

USER_dir = '../user_dir'
import sys
sys.path.append(USER_dir)

***
<a id="problem_statement"></a>
# 2. Problem statement

The numerical example models a simple cantilever beam with length, $L$, distance to the neutral axis $y$, Young’s modulus, $E$, second moment of inertia, $I$, and external load, $F$. We will compute the bending stress $σ$, and the deflection $d$, assuming the above input parameters are **uncertain numbers**.

We will use this example throughout the document to illustrate the differences among various uncertainty quantification approaches. A complete description of the following methods can be found in the [DAWS1 report](https://sites.google.com/view/dawsreports/up).

![alt text](../assets/cantilever.png)

**Fig.1** Cantilever beam with input parameters.  

***
<a id="create_propagating_function"></a>
# 3. Create the propagating function


A propagating function is created which calculates the deflection, $d$, as a function of $L$, $E$, $I$, and $F$.
- The function's input is only the uncertain numbers.
- As sampling techniques are used it is likely that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this.


Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.

In [4]:
def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:
        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
        deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    except:
        deflection = np.nan

    return deflection

In [5]:
# know the function with default Python help mechanism
cantilever_beam_deflection?

Signature: cantilever_beam_deflection(x)
Docstring:
Calculates deflection and stress for a cantilever beam.

Args:
    x (np.array): Array of input parameters:
        x[0]: Length of the beam (m)
        x[1]: Second moment of area (mm^4)
        x[2]: Applied force (N)
        x[3]: Young's modulus (MPa)

Returns:
    float: deflection (m)
           Returns np.nan if calculation error occurs.
File:      c:\users\ioanna\appdata\local\temp\ipykernel_32232\2799985209.py
Type:      function

In [10]:
# Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.
from PyUncertainNumber.UP.performance_func import cb_deflection, cb_stress

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Alternatively, such propating function can be imported as a Python function/method obj     │
│ ❱ 2 from PyUncertainNumber.UP.performance_func import cb_deflection, cb_stress                   │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'cb_deflection' from 'PyUncertainNumber.UP.performance_func' 
(C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\UP\performance_func.py)

## Verify the function

To ensure that the function yields meaningful results. We consider that input has the nominal values seen below.

The function should yield deflection equal to 0.162m.
 

In [11]:
# test the function

y  = 0.155 # m
L  = 10.05 # m
I = 0.000386 # m**4
F = 37 # kN
E = 200 # GPa

x = np.array([L, I, F, E])
deflection = cantilever_beam_deflection(x)

print(deflection) # 0.162m

0.1621665786917099


***
<a id="propagate_epistemic_uncertainty"></a>
# 4. Epistemic uncertainty

<a id="4.1.-uncertainty-characterisation"></a>

## 4.1. Uncertainty Characterisation

Construct `UncertainNumbers` objects for the inputs assuming they are intervals with the lower and upper values are seen below

- $y = [0.145, 0.155] \ m$

- $L = [9.95, 10.05] \ m$

- $I = [0.0003861591, 0.0005213425] \ m^{4}$

- $F = [11, 37] \ kN$

- $E = [200, 220] \ GPa$
 

In [13]:
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds=[9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds=[0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds=[11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])

<a id="4.2-uncertainty-propatation-methods"></a>

## 4.2 Ucertainty propagation methods 

Choose from a suite of black box propagating techniques to propagate the intervals through the model. 

<a id="4.2.1.-Endpoints-propatation"></a>

### 4.2.1. Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of $n = 2^{d}$. 

For the working example, there are d = 5 intervals which results in $n = 2^{5} = 32$ input combinations.

#### Assumptions




- [x] Ioanna's original top-level UP function with  the 'endspoints' method

In [ ]:
''' original implementation from Ioanna: to be deleted'''

L =  np.array([9.95, 10.05]) # m
I =  np.array([0.0003861591, 0.0005213425])# m**4
F =  np.array([11, 37]) # kN
E =  np.array([200, 220]) # GPa

# Create a 2D np.array with all uncertain input parameters in the **correct** order.
xInt = np.array([L, I, F, E])
print(xInt)

''' naive imple of hint on combination number of endpoints method '''
# # How many input combinations are expected from the endpoint propagation?
# d = xInt.shape[0] # is the number of uncertain input expressed as intervals.
# n = 2**d # The total number of combinations 
# print("Total number of input combinations for the endpoint method:", n) 

# method = "endpoint"
# base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
# min_y, max_y, x_miny, x_maxy = main.up_bb(xInt, cantilever_beam_deflection, n = None, method= method, save_raw_data = "no", base_path = base_path)

# #TODO envelope the results of min_y nad max_y 
# # plot the extremities for each input variable
# print(min_y)
# print(max_y)

METHOD = "endpoint"
base_path = ""

# TODO return either as namedTuple or dict to be explicit
a = up_bb(xInt, 
          cantilever_beam_deflection, 
          n = None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = base_path)
              

- [x] Leslie's standalone endspoints implementation

In [ ]:
''' works perfect '''

# deflection_bounds_e = UN.endpointsMethod(vars=['L', 'I', 'F', 'E'], 
#                                          func = cb_deflection, 
#                                          name='deflection', 
#                                          symbol='D')
# deflection_bounds_e

# stress_bounds_e = UN.endpointsMethod(vars=['y', 'L', 'I', 'F'], 
#                                      func= cb_stress, 
#                                      name='stress', 
#                                      symbol='S')
# stress_bounds_e

- [x] Leslie's integrated top-lelvel endspoints implementation

In [14]:
''' now the top-level `up_bb()` takes either nd.array or a list of UN objects; '''

METHOD = "endpoint"

# TODO return either as namedTuple or dict to be explicit

a = up_bb(vars=['L', 'I', 'F', 'E'], 
          fun=cantilever_beam_deflection, 
          n = None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

Total number of input combinations for the endpoint method: 16


Evaluating samples: 100%|██████████| 16/16 [00:00<?, ?it/s]

There are no NA values produced by the input


In [15]:
# returned a dict that contains the UN object and some other information
a.keys()

dict_keys(['min', 'max', 'bounds', 'raw_data', 'un'])

<a id="4.2.2.-Subinterval-reconstitution-propagation"></a>

### 4.2.2. Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [16]:
METHOD = "subinterval"
base_path = ""

# TODO return either as namedTuple or dict to be explicit
a = up_bb(vars=['L', 'I', 'F', 'E'], 
          fun = cantilever_beam_deflection, 
          n = 10, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = base_path)

Total number of input combinations for the subinterval method: 14641


Evaluating samples: 100%|██████████| 14641/14641 [00:00<?, ?it/s]


There are no NA values produced by the input


In [17]:
a.keys()

dict_keys(['min', 'max', 'bounds', 'raw_data', 'un'])

<a id="4.2.3.-Sampling-propagation"></a>

### 4.2.3. Sampling propagation

- Brute Monte Carlo
- Latin Hypercube
- Brute Monte Carlo + endspoints
- Latin Hypercuve + endpoints

In [ ]:
METHOD = "montecarlo"
base_path = ""

a = up_bb(vars=['L', 'I', 'F', 'E'], 
          fun = cantilever_beam_deflection, 
          n = 10, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

<a id="4.2.4.-Optimisation-methors"></a>

### 4.2.4. Optimisation methods

- Local optimisation
- Genetic algorithm


In [22]:
''' so no save data for this method !? '''

METHOD = "local_optimisation"

a = up_bb(vars=['L', 'I', 'F', 'E'], 
       fun = cantilever_beam_deflection, 
       x0 = None, 
       method = METHOD, 
       method_loc = 'Nelder-Mead')

In [19]:
a.keys()

dict_keys(['un', 'bounds', 'min', 'max'])

### Naive interval arithmetics

In [ ]:
defl = cantilever_beam_deflection(L, I, F, E)

In [ ]:
defl

***
<a id="aleatory_uncertainty_propagation"></a>
# 5. Aleatory uncertainty

when inputs have various types of uncertainty.

In [ ]:
#TODO could Scott elaborate on envelop?

***
<a id="mixed_uncertainty_propagation"></a>
# 6. Mixed types of uncertainty

when inputs have various types of uncertainty.